<a href="https://colab.research.google.com/github/midnightripper/accuracy_improvement/blob/main/VAE_Changing_all_the_Variances.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


###Prerequisite

In [2]:
pip install tensorflow-addons

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 591.0/591.0 kB 14.8 MB/s eta 0:00:00


In [3]:
import numpy as np
import tensorflow as tf
import tensorflow_addons as tfa
import scipy.io as sio
import pandas as pd
import matplotlib.pyplot as plt

from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.models import Model
from sklearn.preprocessing import StandardScaler
from keras.models import Sequential
from keras.layers import Activation
from keras.layers import Input
from keras.layers import Reshape
from tensorflow.keras import backend as K

/usr/local/lib/python3.9/dist-packages/tensorflow_addons/utils/tfa_eol_msg.py:23: UserWarning: 

TensorFlow Addons (TFA) has ended development and introduction of new features.
TFA has entered a minimal maintenance and release mode until a planned end of life in May 2024.
Please modify downstream libraries to take dependencies from other repositories in our TensorFlow community (e.g. Keras, Keras-CV, and Keras-NLP). 

For more information see: https://github.com/tensorflow/addons/issues/2807 

  warnings.warn(


Load the train and test data splits

In [4]:
GER_test = (np.array(sio.loadmat('/content/drive/MyDrive/training data/GER_test.mat')['GER_test'])).transpose()
GER_train = (np.array(sio.loadmat('/content/drive/MyDrive/training data/GER_train.mat')['GER_train'])).transpose()
test_frame = pd.DataFrame(GER_test).sort_values(1)
train_frame = pd.DataFrame(GER_train).sort_values(1)

In [5]:
print(f"x_train shape: {GER_train.shape} - y_train shape: {GER_train.shape}")
print(f"x_test shape: {GER_test.shape} - y_test shape: {GER_test.shape}")

x_train shape: (6981, 21) - y_train shape: (6981, 21)
x_test shape: (6248, 21) - y_test shape: (6248, 21)


In [6]:
def get_data_labels(split):
    x = pd.DataFrame(split)
    labels = x[0].values.astype(np.uint8)
    del x[0],x[1]
    data = x.values 
    return data, labels
    
x_train, y_train = get_data_labels(train_frame.values.tolist())
x_test, y_test = get_data_labels(test_frame.values.tolist())

In [7]:
df = pd.DataFrame(x_train)
scaler = StandardScaler()
x_train = pd.DataFrame(scaler.fit_transform(df), columns=df.columns)

In [8]:
df2 = pd.DataFrame(x_test)
scaler = StandardScaler()
x_test = pd.DataFrame(scaler.fit_transform(df2), columns=df.columns)

In [9]:
print(x_train.mean())

0     5.699813e-17
1    -1.628518e-17
2     1.791370e-16
3     8.142590e-18
4    -4.071295e-18
5    -4.274860e-16
6    -8.676947e-17
7     8.549719e-17
8     7.302885e-17
9    -4.885554e-17
10    9.771108e-17
11    3.257036e-17
12    1.302814e-16
13    8.386867e-16
14    1.526736e-18
15    2.157786e-16
16    2.015291e-16
17    9.440315e-17
18    8.193481e-17
dtype: float64


In [10]:
print(x_train.std())

0     1.000072
1     1.000072
2     1.000072
3     1.000072
4     1.000072
5     1.000072
6     1.000072
7     1.000072
8     1.000072
9     1.000072
10    1.000072
11    1.000072
12    1.000072
13    1.000072
14    1.000072
15    1.000072
16    1.000072
17    1.000072
18    1.000072
dtype: float64


In [11]:
print(f"Size of the training set: {len(x_train)}")

Size of the training set: 6981


In [12]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state=42)

print(f"Size of the x training set: {len(x_train)}")
print(f"Size of the x validation set: {len(x_val)}")
print(f"Size of the y training set: {len(y_train)}")
print(f"Size of the y validation set: {len(y_val)}")

Size of the x training set: 5584
Size of the x validation set: 1397
Size of the y training set: 5584
Size of the y validation set: 1397


In [13]:
print(x_train[:5])
print(y_train[:5])

            0         1         2         3         4         5         6   \
6565 -1.076680 -1.065748 -1.012843 -0.799906 -0.713923 -0.710367 -1.932554   
4003  0.582947  1.135969  1.297658  1.015694  0.868769 -0.130759 -1.806106   
1010 -0.545586 -0.653364 -0.430347 -0.782925 -0.775639 -0.323475 -0.652264   
4617 -0.566326 -0.578170 -0.686170 -0.379677 -0.210357 -1.287966  0.392749   
931   0.245015  0.061639  0.443537 -0.616326 -0.935091  0.915754  0.721268   

            7         8         9         10        11        12        13  \
6565  2.396384 -0.985383 -0.853229 -0.838086 -0.790418 -0.770441  1.082992   
4003  1.601756 -0.026166  1.974222  2.026149  2.399590  2.084844 -0.682288   
1010  0.034567 -1.103360  0.015721  0.285905  0.997402  0.964844 -2.012156   
4617 -0.268374 -0.030353 -0.463698 -0.261271  1.170901  1.321986 -2.641625   
931  -0.828500 -0.803408 -0.062883 -0.065124 -0.179201 -0.313890  0.995570   

            14        15        16        17        18  
6565 

Early stopping to prevent over-fitting

In [14]:
es = tf.keras.callbacks.EarlyStopping(monitor='val_loss', mode='min', verbose=1, patience=5)





# Function Calls

In [15]:
input_shape = (19,)

In [16]:
def sampling(args):
    """Reparameterization trick. Instead of sampling from Q(z|X), 
    sample eps = N(0,I) z = z_mean + sqrt(var)*eps.

    Parameters:
    -----------
    args: list of Tensors
        Mean and log of variance of Q(z|X)

    Returns
    -------
    z: Tensor
        Sampled latent vector
    """

    z_mean, z_log_var = args
    eps = tf.random.normal(tf.shape(z_log_var), dtype=tf.float32, mean=0., stddev=0.004 , name='epsilon')
    z = z_mean + tf.exp(z_log_var / 2) * eps
    return z

In [17]:
inputs = keras.layers.Input(shape=input_shape, name='input')
x = keras.layers.Dense(64, activation='relu')(inputs)
z_mean = keras.layers.Dense(19, name='z_mean')(x)
z_log_var = keras.layers.Dense(19, name='z_log_var')(x)

In [18]:
z = keras.layers.Lambda(sampling, name='z')([z_mean, z_log_var])
encoder = keras.Model(inputs, [z_mean, z_log_var, z], name='encoder')
encoder.summary()

Model: "encoder"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 19)]         0           []                               
                                                                                                  
 dense (Dense)                  (None, 64)           1280        ['input[0][0]']                  
                                                                                                  
 z_mean (Dense)                 (None, 19)           1235        ['dense[0][0]']                  
                                                                                                  
 z_log_var (Dense)              (None, 19)           1235        ['dense[0][0]']                  
                                                                                            

# Decoder

In [19]:
#Giving input to decoder and initialising shape
latent_inputs = keras.layers.Input(shape=(19), name='z_sampling')
middle=keras.layers.Dense(64, activation='relu')(latent_inputs)
x = keras.layers.Dense(19, activation='relu')(middle)

# instantiate decoder model
decoder = keras.Model(latent_inputs, x, name='decoder')

# Implementing VAE without Loss

In [20]:
outputs = decoder(encoder(inputs)[2])  # Select the Z value from outputs of the encoder
vae = keras.Model(inputs, outputs, name='vae')
vae.summary()

Model: "vae"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input (InputLayer)          [(None, 19)]              0         
                                                                 
 encoder (Functional)        [(None, 19),              3750      
                              (None, 19),                        
                              (None, 19)]                        
                                                                 
 decoder (Functional)        (None, 19)                2515      
                                                                 
Total params: 6,265
Trainable params: 6,265
Non-trainable params: 0
_________________________________________________________________


# Initialising Loss 

In [21]:
autoencoder = Model(inputs, outputs)

In [22]:
reconstruction_loss = K.mean(K.square(outputs - inputs), axis=-1)
kl_loss = -0.5 * K.sum(1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
vae_loss = K.mean(reconstruction_loss + kl_loss)


# Training VAE

In [23]:
autoencoder.add_loss(vae_loss)
autoencoder.summary()
autoencoder.compile(optimizer='adam')

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input (InputLayer)             [(None, 19)]         0           []                               
                                                                                                  
 encoder (Functional)           [(None, 19),         3750        ['input[0][0]']                  
                                 (None, 19),                                                      
                                 (None, 19)]                                                      
                                                                                                  
 decoder (Functional)           (None, 19)           2515        ['encoder[0][2]']                
                                                                                              

In [24]:
autoencoder.fit(x_train, epochs=50, batch_size=32, validation_data=(x_val, ),callbacks=[es])

Epoch 1/50
175/175 [==============================] - 2s 4ms/step - loss: 1.5707 - val_loss: 0.9775
Epoch 2/50
175/175 [==============================] - 0s 3ms/step - loss: 0.8246 - val_loss: 0.7140
Epoch 3/50
175/175 [==============================] - 0s 3ms/step - loss: 0.6528 - val_loss: 0.5945
Epoch 4/50
175/175 [==============================] - 0s 3ms/step - loss: 0.5654 - val_loss: 0.5347
Epoch 5/50
175/175 [==============================] - 0s 3ms/step - loss: 0.5182 - val_loss: 0.4968
Epoch 6/50
175/175 [==============================] - 1s 3ms/step - loss: 0.4908 - val_loss: 0.4730
Epoch 7/50
175/175 [==============================] - 0s 2ms/step - loss: 0.4697 - val_loss: 0.4540
Epoch 8/50
175/175 [==============================] - 0s 2ms/step - loss: 0.4551 - val_loss: 0.4432
Epoch 9/50
175/175 [==============================] - 1s 3ms/step - loss: 0.4455 - val_loss: 0.4319
Epoch 10/50
175/175 [==============================] - 1s 4ms/step - loss: 0.4361 - val_loss: 0.4265